In [1]:
from pynq import Overlay
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from os import path
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import pickle as pkl
%matplotlib inline

In [2]:
class TestChip(Overlay):
    """TestChip class is the main driver
    class for interacting with our FPGA bitstream
    Coded by: Uncle Arash
    Version: 0.0
    """
    def __init__(self, ol_path, **kwargs):
        super().__init__(ol_path)
        self.heater_base_address = 0x00000000
        self.RO_base_address = 0x00000000
        self.BTI_base_write_address = 0x00000000
        self.BTI_base_read_address = 0x00000004
        self.temp_sensor_address = 0x200
        self.counter_address_increament = 0x04
        self.num_oscillators = 31
        self.num_BTI = 31
        self.intensity_dict = {i: int(sum(
                                          [2**j for j in range(i)])
                                      ) for i in range(1, 33)
                               }
        self.intensity_dict[0] = 0
        self.sensor_dict = {i: 2**i for i in range(32)}
        for key, value in kwargs.items():
            if "heater_base_address" in key:
                self.heater_base_address = value
            if "counter_base_address" in key:
                self.counter_base_address = value
            if "num_oscillators" in key:
                self.num_oscillators = value
            if "counter_address_increament" in key:
                self.counter_address_increament = value
            if "temp_sensor_address" in key:
                self.temp_sensor_address = value
        self._a = 4.07548611   # 5
        self._b = 0.50103761   # 50
        self.temp_ctrl_sensitivity = 2
        self.temp_ctrl_intensity = 0

    def XADC_temp(self):
        return ((self.Temp_sensor.read(self.temp_sensor_address
                                       ) >> 4) * 503.975/4096 - 273.15)

    def freq2temp(self, Δf):
        """The values of a and b are for 5 stage ROs in ZYNQ 7000"""
        return Δf * self._a + self._b

    def read_RO(self, RO_list):
        """Reads the frequency of selected ROs
        Parameters:
        RO_list (list of int): list of ROs whose values we read

        Returns: freq_list (nparray)
        """
        len_ro = len(RO_list)
        freq_list = np.zeros((len_ro))
        for i in range(len_ro):
            assert RO_list[i] <= self.num_oscillators
            freq_list[i] = self.RO0.read(
                self.RO_base_address +
                RO_list[i] * self.counter_address_increament
            )/1000
        return freq_list

    def read_multi_RO(self, RO_dict):
        """Reads the frequency of selected ROs
        Parameters:
        RO_dict : {keys=RO_ip_name, values=[RO list per IP]}

        Returns: freq_dict {keys=RO_ip_name, values=[frequencies (nparray)]}
        """
        freq_dict={}
        for item in RO_dict.keys():
            RO_list = RO_dict[item]
            len_ro = len(RO_list)
            freq_list = np.zeros((len_ro))
            RO = getattr(self, item)
            for i in range(len_ro):
                freq_list[i] = RO.read(
                    self.RO_base_address +
                    RO_list[i] * self.counter_address_increament
                )/1000
            freq_dict[item] = freq_list
        return freq_dict

    def read_BTI(self, BTI_list):
        """Reads the frequency of selected BTI sensort
        Parameters:
        BTI_list (list of int): list of BTI sensors whose values we read

        Returns: freq_list (nparray)
        """
        len_ro = len(BTI_list)
        freq_list = np.zeros((len_ro))
        for i in range(len_ro):
            assert BTI_list[i] <= self.num_BTI
            #   Putting the BTI sensors into the counting mode
            self.BTI0.write(self.BTI_base_write_address, self.sensor_dict[BTI_list[i]])
            freq_list[i] = self.BTI0.read(
                self.BTI_base_read_address +
                BTI_list[i] * self.counter_address_increament
            )/1000
            #   Putting the BTI sensors back into the aging mode
            self.BTI0.write(self.BTI_base_write_address, 0)
        return freq_list
    
    def read_multi_BTI(self, BTI_dict):
        """Reads the frequency of selected BTI sensort
        Parameters:
        BTI_dict: {keys=BTI_ip_name, values=[BTI list per IP]}

        Returns: freq_dict {keys=BTI_ip_name, values=[frequencies (nparray)]}
        """
        freq_dict={}
        for item in BTI_dict.keys():
            BTI_list = BTI_dict[item]
            len_ro = len(BTI_list)
            freq_list = np.zeros((len_ro))
            BTI = getattr(self, item)
            for i in range(len_ro):
                #   Putting the BTI sensors into the counting mode
                BTI.write(self.BTI_base_write_address, self.sensor_dict[BTI_list[i]])
                freq_list[i] = BTI.read(
                    self.BTI_base_read_address +
                    BTI_list[i] * self.counter_address_increament
                )/1000
                #   Putting the BTI sensors back into the aging mode
                BTI.write(self.BTI_base_write_address, 0)
        return freq_dict

    def top_region_heat_on(self, intensity):
        """Turns the top region heat on
        top region heat, heats up the whole top region of the chip

        Parameters:
        intensity (int): intensity of the heat
        (the final temperature depends on ventilation
        and/or isolation of the chip)
        intensity has to be between 0 to 64

        Returns: None
        """

        if intensity > 64:
            intensity = 64
        elif intensity < 0:
            intensity = 0

        if intensity < 33:
            lsb_heater = self.intensity_dict[intensity]
            msb_heater = 0x00000000
        else:
            lsb_heater = 0xFFFFFFFF
            msb_heater = self.intensity_dict[intensity-32]

        self.heater.write(self.heater_base_address, lsb_heater)
        self.heater.write(self.heater_base_address+0x04, msb_heater)

    def top_region_heat_off(self):
        """Turns the top region heat off
        top region heat, heats up the whole top region of the chip

        Returns: None
        """

        self.heater.write(self.heater_base_address, 0x00000000)
        self.heater.write(self.heater_base_address+0x04, 0x00000000)

    def fix_temperature(self, desired_temperature):
        """simple control scheme to fix the temperature to a desired value

        Returns: None
        """

        if self.XADC_temp() > (desired_temperature +
                               self.temp_ctrl_sensitivity):
            self.temp_ctrl_intensity -= 1
            self.top_region_heat_on(self.temp_ctrl_intensity)
        elif self.XADC_temp() < (desired_temperature -
                                 self.temp_ctrl_sensitivity):
            self.temp_ctrl_intensity += 1
            self.top_region_heat_on(self.temp_ctrl_intensity)
        if self.temp_ctrl_intensity > 64:
            self.temp_ctrl_intensity = 64
        elif self.temp_ctrl_intensity < 0:
            self.temp_ctrl_intensity = 0

    def __str__(self):
        return (f"Number of ROs: {self.num_oscillators}; "
                "Current temperature: {self.XADC_temp()}; "
                "Number of BTIs: {self.num_BTI}")

In [7]:
def record(ol, total_duration, every, num_oscillators):
    """
    parameters
    ----------
    total_duration : total duration in seconds
    every : sampling step size in seconds
    """
    data = []   # [RO0, ..., RO{num_oscillators},Temperature, En_Freq, Duty_Cycle]
    RO_dict = dict(RO0=list(range(3)), BTI0=list(range(3)), BTI1=list(range(3)), RO3=list(range(3)), RO4=list(range(3)))
    times = []
    init_time = datetime.now()
    now_time = init_time
    desired_temperature = 85
    while(now_time < (init_time + total_duration)):
        now_time = datetime.now()
        temp = ol.XADC_temp()
        ####temperature controller starts
        #ol.fix_temperature(desired_temperature)
        ####temperature controller ends
        output_dict = ol.read_multi_RO(RO_dict)
        current_read = np.hstack((output_dict['RO0'], output_dict['BTI0']))
        current_read = np.hstack((current_read, output_dict['BTI1']))
        current_read = np.hstack((current_read, output_dict['RO3']))
        current_read = np.hstack((current_read, output_dict['RO4']))
        current_read = np.hstack((current_read, np.array([temp, 0, 0])))
        now_pacific_live = datetime.now(timezone('US/Pacific'))
        times.append(now_pacific_live)
        data.append(current_read)
        while(datetime.now() < now_time + every):
            pass
        print(temp)
    data = np.vstack(data)
    output = pd.DataFrame(data, columns=([f'RO{i}' for i in range(num_oscillators)] + ['Temperature', 'En_Freq', 'Duty_Cycle']))
    output['Timestamp'] = pd.DataFrame(dict(Timestamp=times))
    return output

In [9]:
now_pacific = datetime.now(timezone('US/Pacific'))
every = timedelta(seconds = 1)
total_duration = timedelta(seconds = 10)
mail_content = '''Hello,
    This is a test mail.
    In this mail we are sending some attachments.
    The mail is sent using Python SMTP library.
    Thank You
    '''
#The mail addresses and password
sender_address = 'projectms555@gmail.com'
sender_pass = 'projectms555!*'
receiver_address = 'projectms555@gmail.com'
    
for i in range(0, 1):
    ol = TestChip(f'/home/xilinx/pynq/overlays/simple_ro_during_stress/design_1.bit')
    output = record(ol, total_duration, every, 15)
    output.to_pickle(f'./Parvez_RO1_during_{i}.pkl')
    with open(f'./Parvez_RO1_during_{i}.pkl', "rb") as pkl_file:
        object = pkl.load(pkl_file)
    df = pd.DataFrame(object)
    df.to_csv(f'./Parvez_RO1_during_{i}.csv')
    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'A Time ' + str(i) + 'hr test mail sent by Python. It has an attachment.'
    #The subject line
    #The body and the attachments for the mail
    message.attach(MIMEText(mail_content, 'plain'))
    attach_file_name = f'./Parvez_RO1_during_{i}.pkl'
    attach_file = open(attach_file_name, 'rb') # Open the file as binary mode
    payload = MIMEBase('application', 'octate-stream')
    payload.set_payload((attach_file).read())
    encoders.encode_base64(payload) #encode the attachment
    #add payload header with filename
    payload.add_header('Content-Decomposition', 'attachment', filename='instagram')
    message.attach(payload)
    #Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()
    print('Mail Sent')

    
##### Replace the overlay
ol = TestChip(f'/home/xilinx/pynq/overlays/adder/adder.bit')
##### Replace the overlay

45.40255737304693
46.14080200195315
46.01776123046881
45.77167968750001
46.386883544921886
46.01776123046881
46.01776123046881
46.63296508789068
46.386883544921886
46.263842773437545
46.63296508789068
Mail Sent


In [5]:
ol = TestChip(f'/home/xilinx/pynq/overlays/adder/adder.bit')